### 9.1 Wrapper

In [1]:
import time
from functools import wraps
from inspect import signature
import types
from collections import OrderedDict

In [2]:
def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [3]:
@timeit
def countdown(n:int):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1
        
countdown(100000)
countdown(1000)
        

countdown 0.012696027755737305
countdown 0.00013709068298339844


In [4]:
countdown.__name__, countdown.__doc__, countdown.__annotations__, countdown.__wrapped__

('countdown',
 '\n    Counts down\n    ',
 {'n': int},
 <function __main__.countdown(n: int)>)

In [5]:
print(signature(countdown))

(n: int)


### 9.4 Decorator with arguments

In [6]:
import logging
logging.basicConfig(level=logging.WARNING)

In [7]:
from functools import partial

# Utility decorator to attach a func as an attribute of an obj
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj) # This is the wrapper around the func -> attach_wrapper(obj)(func)
    setattr(obj, func.__name__, func)
    return func

In [8]:
def logged(level, name=None, message=None):
    
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
    
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
            
        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg
            
        return wrapper
        
    return decorate

In [9]:
@logged(logging.WARNING)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

In [10]:
spam()
add(3, 4)

CRITICAL:example:spam


Spam!


7

In [11]:
spam.set_level(logging.ERROR)
spam.set_message('Custom msg')
spam()

ERROR:example:Custom msg


Spam!


### 9.7 Type Assertion

In [12]:
def spam(x, y, z=42):
    pass

In [13]:
sig = signature(spam)
print(sig)

(x, y, z=42)


In [14]:
sig.parameters

mappingproxy({'x': <Parameter "x">,
              'y': <Parameter "y">,
              'z': <Parameter "z=42">})

In [15]:
sig.parameters['z'].name, sig.parameters['z'].default, sig.parameters['z'].kind

('z', 42, <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>)

In [16]:
bound_types = sig.bind_partial(int, z=int)
bound_types.arguments

OrderedDict([('x', int), ('z', int)])

In [17]:
def typeassert(*ty_args, **ty_kwargs):
    
    def decorate(func):
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs).arguments
            # Enforce type assertions
            for name, value in bound_values.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError('Argument {} must be of type {}'.format(name, bound_types[name]))
            return func(*args, **kwargs)
        return wrapper
    return decorate
            
            

In [18]:
@typeassert(int, int)
def add(x, y=4):
    return x + y

@typeassert(y=str)
def printer(x, y, z):
    print(x, y, z)

print('2 + 3 = {}'.format(add(2, 3)))
printer(1, 'world', 3)
try:
    add('bat', 'man')
except TypeError as err:
    print('Error: {}'.format(err))
    
try:
    printer(1, 2, 3)
except TypeError as err:
    print('Error: {}'.format(err))

2 + 3 = 5
1 world 3
Error: Argument x must be of type <class 'int'>
Error: Argument y must be of type <class 'str'>


### 9.9 Decorator as a Class

In [19]:
class Profiled:
    
    def __init__(self, func):
        print('__init__ called')
        wraps(func)(self)
        self.ncalls = 0
        
    def __call__(self, *args, **kwargs):
        print('__call__ called')
        self.ncalls += 1
        return self.__wrapped__(*args, **kwargs)
    
    def __get__(self, instance, cls):
        print('__get__ called')
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)
    

In [20]:
@Profiled
def sub(x, y):
    return x-y

__init__ called


In [21]:
print(sub(20,3))
sub.ncalls

__call__ called
17


1

In [22]:
class Spam:
    @Profiled
    def bar(self, x):
        print(x)
    
    def foo(self, y):
        print(y)

__init__ called


In [23]:
s = Spam()
s.bar('author')
s.bar('again')
s.bar.ncalls

__get__ called
__call__ called
author
__get__ called
__call__ called
again
__get__ called


2

In [24]:
def profiled(func):
    ncalls = 0
    @wraps(func)
    def wrapped(*args, **kwargs):
        nonlocal ncalls
        ncalls += 1
        return func(*args, **kwargs)
    wrapped.ncalls = lambda : ncalls
    return wrapped

In [25]:
@profiled
def mul(x, y):
    return x*y

mul(2, 3)
mul(3, 4)
mul.ncalls()

2

### 9.11 Decorators to add arguments to functions

In [26]:
def optional_debug(func):
    @wraps(func)
    def wrapped(*args, debug=False, **kwargs):
        if debug:
            print('Calling {}'.format(func.__name__))
        return func(*args, **kwargs)
    return wrapped

In [27]:
@optional_debug
def spam(x, y, z):
    print(x, y, z)

In [28]:
spam(1, 2, 3)

1 2 3


In [29]:
spam(3, 4, 5, debug=True)

Calling spam
3 4 5


### 9.13 MetaClass - Class Creation

In [30]:
class NoInstances(type):
    def __call__(cls, *args, **kwargs):
        raise RuntimeError('Can not instantiate {}'.format(cls))
        
class Spam(metaclass=NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok')
        

In [31]:
Spam.grok('author')
try:
    s = Spam('Guido')
except RuntimeError as err:
    print(err)

Spam.grok
Can not instantiate <class '__main__.Spam'>


In [32]:
class Singleton(type):
    def __init__(cls, name, bases, dct):
        cls.__instance = None
        super().__init__(name, bases, dct)
        
    def __call__(cls, *args, **kwargs):
        if cls.__instance is None:
            cls.__instance =  super().__call__(*args, **kwargs)
        return cls.__instance
        
            
class SingleSpam(metaclass=Singleton):
    def __init__(self):
        print('Creating SingleSpam')

In [33]:
s = SingleSpam()
q = SingleSpam()
s is q

Creating SingleSpam


True

### 9.14 Class Attribute Definition Order

In [34]:
class Typed:
    _expected_type = None
    def __init__(self, name=None):
        self._name = name
        
    def __set__(self, instance, value):
        if not isinstance(value, self._expected_type):
            raise TypeError('Expected {!s} for {}'.format(self._expected_type, self._name))
        instance.__dict__[self._name] = value
        
        

In [35]:
class Integer(Typed):
    _expected_type = int
    
class Float(Typed):
    _expected_type = float
    
class String(Typed):
    _expected_type = str

In [36]:
class OrderedMeta(type):
    @classmethod
    def __prepare__(cls, clsname, bases):
        return OrderedDict()
    
    def __new__(meta, clsname, bases, clsdct):
        d = dict(clsdct)
        order = []
        for name, value in clsdct.items():
            if isinstance(value, Typed):
                value._name = name
                order.append(name)
        d['_order'] = order
        return super().__new__(meta, clsname, bases, d)
                

In [37]:
class Structure(metaclass=OrderedMeta):
    def to_csv(self):
        return ','.join(str(getattr(self, name)) for name in self._order)
    
class Stock(Structure):
    name = String()
    shares = Integer()
    price = Float()
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [38]:
try:
    goog = Stock('GOOG', 10, 1500)
except TypeError as err:
    print(err)
amazon = Stock('AMAZN', 10, 2000.)
amazon.to_csv()

Expected <class 'float'> for price


'AMAZN,10,2000.0'

### 9.16 Enforcing argument signature

In [39]:
import inspect
from inspect import Signature, Parameter

In [40]:
temp_parms = [ Parameter('x', Parameter.POSITIONAL_OR_KEYWORD), 
               Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42), 
               Parameter('z', Parameter.KEYWORD_ONLY, default=None) ]
temp_sig = Signature(temp_parms)
print(temp_sig)

(x, y=42, *, z=None)


In [41]:
def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD) for name in names]
    return Signature(parms)

In [42]:
class Structure:
    __signature__ = make_sig()
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values:
            setattr(self, name, value)
            
            
class Point(Structure):
    __signature__ = make_sig('x', 'y')

In [43]:
print(inspect.signature(Point))

(x, y)


### 9.17 Coding Conventions in Classes

In [44]:
class NoMixedCaseMeta(type):
    def __new__(meta, clsname, bases, clsdct):
        for name in clsdct:
            if name.lower() != name:
                raise TypeError('Bad attribute name: {}'.format(name))
        return super().__new__(meta, clsname, bases, clsdct)    

In [45]:
class Root(metaclass=NoMixedCaseMeta):
    pass

class A(Root):
    def foo_bar(self):
        pass

In [46]:
try:
    class B(Root):
        def FooBar(self):
            pass
except TypeError as err:
    print(err)

Bad attribute name: FooBar


In [47]:
class MatchSignaturesMeta(type):
    def __init__(cls, clsname, bases, clsdct):
        super().__init__(clsname, bases, clsdct)
        sup = super(cls, cls)
        for name, value in clsdct.items():
            if name.startswith('_') or not callable(value):
                continue
            prev_dfn = getattr(sup, name, None)
            if prev_dfn:
                prev_sig = inspect.signature(prev_dfn)
                curr_sig = inspect.signature(value)
                if prev_sig != curr_sig:
                    logging.warning('Signature mismatch in {}. {} != {}'.format(value.__qualname__, curr_sig, prev_sig))
                   

In [48]:
class Root(metaclass=MatchSignaturesMeta):
    pass

class A(Root):
    def foo(self, x, y):
        pass
    
    def spam(self, x, *, z):
        pass
    
class B(A):
    def foo(self, a, b):
        pass
    
    def spam(self, x, z):
        pass